In [1]:
import os
import zipfile
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import numpy as np
from itertools import cycle
from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp
from sklearn.metrics import roc_auc_score

In [2]:
train_datagen = ImageDataGenerator(rescale=1/255)
validation_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
    "training/MultiClassification/train",
    classes = ["humidity", "none", "scale"],
    target_size=(200,200),
    batch_size=120,
    class_mode="categorical",
    shuffle=False)

valid_generator = validation_datagen.flow_from_directory(
    "training/MultiClassification/valid",
    classes = ["humidity", "none", "scale"],
    target_size=(200,200),
    batch_size=120,
    class_mode="categorical",
    shuffle=False)

Found 10652 images belonging to 3 classes.
Found 180 images belonging to 3 classes.


In [3]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(200,200,3)),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(3, activation=tf.nn.sigmoid)
])

In [4]:
model.compile(optimizer=tf.optimizers.Adam(),
              loss="binary_crossentropy",
              metrics=["accuracy"])

In [6]:
history = model.fit(train_generator,
      steps_per_epoch=8,  
      epochs=15,
      verbose=1,
      validation_data = valid_generator,
      validation_steps=8)

Epoch 1/15
8/8 [==============================] - 2s 239ms/step - loss: 27.6597 - accuracy: 0.8427
Epoch 2/15
8/8 [==============================] - 2s 232ms/step - loss: 21.0413 - accuracy: 0.3750
Epoch 3/15
8/8 [==============================] - 2s 235ms/step - loss: 3.8037 - accuracy: 0.7552
Epoch 4/15
8/8 [==============================] - 2s 241ms/step - loss: 5.0962 - accuracy: 0.8823
Epoch 5/15
8/8 [==============================] - 2s 235ms/step - loss: 8.5315 - accuracy: 0.5260
Epoch 6/15
8/8 [==============================] - 2s 234ms/step - loss: 3.4342 - accuracy: 0.6469
Epoch 7/15
8/8 [==============================] - 2s 239ms/step - loss: 3.5393 - accuracy: 0.5573
Epoch 8/15
8/8 [==============================] - 2s 238ms/step - loss: 1.7524 - accuracy: 0.9250
Epoch 9/15
8/8 [==============================] - 2s 240ms/step - loss: 0.3558 - accuracy: 1.0000
Epoch 10/15
8/8 [==============================] - 2s 239ms/step - loss: 1.8374 - accuracy: 0.8646
Epoch 11/15
8/8 [

In [7]:
model.evaluate(valid_generator)

2/2 [==============================] - 0s 71ms/step - loss: 7.3374 - accuracy: 0.4333


[7.33735466003418, 0.4333333373069763]

In [12]:

plt.plot(history)